# Portfolio Risk Report
This notebook demonstrates how to build a Barra-style portfolio snapshot (top holdings, factor exposures, variance decomposition) for a given month-end.

## Workflow
1. Run `python -m src.cli run-pipeline --date YYYY-MM-DD` for the target rebalance.
2. Update `AS_OF` / `TOP_N` below.
3. Execute all cells to obtain portfolio holdings, exposures, and risk contributions.
4. Adapt the summary tables/plots for dashboards or PPT exports.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from src import export_reports
from src.config import ANALYTICS_DB

AS_OF = pd.Timestamp("2025-09-30").date()
TOP_N = 50
plt.style.use("seaborn-v0_8")
ANALYTICS_DB

In [ ]:
holdings = export_reports.load_top_constituents(AS_OF, TOP_N)
holdings["weight"] = holdings["month_end_market_cap"] / holdings["month_end_market_cap"].sum()
holdings.head()

In [ ]:
summary = export_reports.load_portfolio_summary(AS_OF, top_n=TOP_N)
factor_summary = summary[summary["type"] == "factor"].copy()
factor_summary.sort_values("variance_contribution", ascending=False, inplace=True)
factor_summary

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
factor_summary.plot.bar(x="factor", y="variance_contribution", color="#1f77b4", ax=ax, legend=False)
ax.set_title(f"Factor Variance Contributions ({AS_OF}, Top {TOP_N})")
ax.set_ylabel("Variance")
plt.xticks(rotation=45, ha="right")
plt.show()

In [ ]:
aggregate_rows = summary[summary["type"] == "aggregate"]
aggregate_rows

In [ ]:
style_exposures = export_reports.load_style_exposures(AS_OF)
top_exposures = (
    style_exposures.merge(holdings[["gvkey", "weight"]], on="gvkey")
    .assign(weighted_exposure=lambda df: df["exposure"] * df["weight"])
    .sort_values("weighted_exposure", key=lambda s: s.abs(), ascending=False)
)
top_exposures.head(20)